In [28]:
from utils import *

In [29]:
og_path = join(home, "PDMR")
update_path = join("/Users/tushar/PDMR/update")
final_path = join("/Users/tushar/PDMR/")
og_metadata_files = {f: join(og_path, f) for f in get_files(og_path) if f.__contains__('PDMR_metadata')}
new_metadata_files = {f: join(update_path, f) for f in get_files(update_path) if f.__contains__('metadata')}
file_dict = {'PDMR_metadata-cell_model.tsv': 'metadata-cell_model.tsv',
 'PDMR_metadata-model_validation.tsv': 'metadata-model_validation.tsv',
 'PDMR_metadata-patient.tsv': 'metadata-patient.tsv',
 'PDMR_metadata-patient_sample.tsv': 'metadata-sample.tsv',
 'PDMR_metadata-pdx_model.tsv': 'metadata-model.tsv',
 'PDMR_metadata-sharing.tsv': 'metadata-sharing.tsv'}

In [60]:
for og in file_dict.keys():
    if og == "PDMR_metadata-sharing.tsv":
        print(og)
        temp = read_metadata_with_fields(og_metadata_files[og])
        head = temp.iloc[0:4]
        temp = temp.iloc[4:]
        temp2 = read_metadata_with_fields(new_metadata_files[file_dict[og]]).iloc[4:]
        if og.__contains__('patient.tsv'):
            final = pd.concat([head, temp2, temp]).drop_duplicates(subset='patient_id', keep='first')
        elif og.__contains__('model_validation.tsv'):
            final = pd.concat([head, temp2, temp]).drop_duplicates(subset=['validation_technique', 'model_id'], keep='first')
        elif og.__contains__('sharing.tsv'):
            final = pd.concat([head, temp, temp2]).drop_duplicates(subset='model_id', keep='first')
        else:
            final = pd.concat([head, temp2, temp]).drop_duplicates(subset='model_id', keep='first')
        final.to_csv(join(final_path, og), sep='\t',  index=False)

PDMR_metadata-sharing.tsv


In [61]:
import pandas as pd

# Load data into a DataFrame
df = read_metadata_without_fields('/Users/tushar/PDMR/PDMR_metadata-patient_sample.tsv')

# Replace missing values
df['stage'] = df['stage'].fillna('Not provided').str.strip()

# Normalize text
df['stage'] = df['stage'].str.lower()

# Extract key fields using regular expressions
df['Stage_extracted'] = df['stage'].str.extract(r'(stage\s?[ivx]+)', expand=False)
df['Grade_extracted'] = df['stage'].str.extract(r'(high grade|moderately differentiated|poorly differentiated|low grade|grade \d+)', expand=False)
df['Classification'] = df['stage'].str.extract(r'(p?t\d+n\d+m\d+|ajcc|figo)', expand=False)

# Save to a cleaned Excel file
df.to_csv('/Users/tushar/PDMR/PDMR_metadata-patient_sample_extracted.tsv', sep='\t', index=False)


In [104]:
df = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-patient_sample.tsv')
pdmr_specimen = pd.read_csv('/Users/tushar/Downloads/Find Specimen.csv')
treatment = pd.read_csv('/Users/tushar/Downloads/PDM_TreatmentHistoryReport (1).csv')


In [100]:
final = list()
for _, row in df.iterrows():
    if row['collection_method'] == 'Not provided':
        model_id = row['model_id']
        pdmr_row = pdmr_specimen[[True if model_id.__contains__(r) else False for r in pdmr_specimen['model_id']]].reset_index(drop=True)
        if len(pdmr_row)>0:
            collection_method, tumor_site = pdmr_row['TissueType'][0], pdmr_row['BiopsySite'][0]
            row['collection_site'], row['collection_method'] = tumor_site, collection_method
    final.append(row.tolist())
final = pd.DataFrame(final, columns=df.columns)

,Field,patient_id,sample_id,collection_date,collection_event,months_since_collection_1,age_in_years_at_collection,diagnosis,tumour_type,primary_site,...,grade,grading_system,virology_status,gene_mutation_status,sharable,treatment_naive_at_collection,treated_at_collection,treated_prior_to_collection,response_to_treatment,model_id
0,#Description,Unique anonymous/de-identified provider ID for...,Unique ID of the patient tumour sample used to...,Date of collections - if this is not available...,IF DATE ENTERED IN collection_date DO NOT FILL...,IF DATE ENTERED IN collection_date DO NOT FILL...,Patient age at collection. Can be exact age or...,diagnosis at time of collection of the patient...,Collected tumour type.,Site of the primary tumor where primary cancer...,...,The implanded tumour grade value.,Grade classification correspondingly used to d...,Positive virology status at the time of collec...,Outcome of mutational status tests for the fol...,Is patient treatment information available and...,Was the patient treatment naive at the time of...,Was the patient being treated for cancer at th...,Was the patient treated prior to the tumour sa...,Patients response to treatment.,Unique identifier for all the PDXs derived fro...
1,#Example,A0088,CRC0228PRH0000000000D01000,Oct-15,collection 1,0,70,colorectal carcinoma,primary,colon,...,Grade 1,Nottingham grading system,Human Immunodeficiency Virus (HIV),KRAS:G12C,yes,Yes,yes,Yes,Progressive disease,CRC0228PR
2,#Format Requirements,free alphanumerical and -_~.,"unique, free alphanumerical and -_~.",MMM YYYY,"""collection "" + event number",numerical,numerical,Use NCIT ontology terms. Ctrl-Click here to se...,"primary, metastastic, recurrent, or refactory",Use NCIT ontology term where possible. Ctrl-cl...,...,free alphanumerical,free alphanumerical,Comma separated list of virus ontologies from ...,HGNC Symbols,yes/no/not provided,yes/no/not collected/not provided,yes/no/not provided/not collected,yes/no/not collected/ no provided,"RECIST criteria: CR, PR, PD, SD",free alphanumerical and and -_~.
3,#Essential?,essential,essential,if several collections this is essential other...,if several collections this is essential other...,if several collections this is essential other...,essential,essential,essential,essential,...,desirable,desirable,desirable,desirable,essential,essential,desirable,desirable,desirable,essential
4,NaN,523234,523234-163-R,01/06/2020,NaN,NaN,73,Endometrioid endomet. adenocar.,Metastatic,Gynecologic,...,Not provided,Not provided,NaN,NaN,Yes,NaN,NaN,NaN,SD,523234-163-R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,NaN,565248,565248-004-R-J1-PDC,Jan-16,NaN,NaN,55,Lip/oral cavity squam. cell car.,Primary,Head and Neck,...,Intermediate grade,Grade,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,565248-004-R-J1-PDC
1422,NaN,565248,565248-004-R-V1-organoid,Jan-16,NaN,NaN,55,Lip/oral cavity squam. cell car.,Primary,Head and Neck,...,Intermediate grade,Grade,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,565248-004-R-V1-organoid
1423,NaN,648629,648629-189-R,Jul-15,NaN,NaN,82,"Urothelial/bladder cancer, NOS",Primary,Genitourinary,...,High grade,Grade,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,648629-189-R
1424,NaN,476673,476673-061-R-V1-organoid,Mar-16,NaN,NaN,77,Melanoma,Primary,Skin,...,High grade,Grade,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,476673-061-R-V1-organoid


In [109]:
final.to_csv('/Users/tushar/PDMR/PDMR_metadata-patient_sample.tsv', sep='\t', index=False)

In [107]:
final = list()
for _, row in df.iterrows():
    if _ > 3:
        patient_id = row['patient_id']
        pdmr_row = treatment[[True if patient_id == r else False for r in treatment['Patient ID']]].reset_index(drop=True)
        if len(pdmr_row)>0:
            regimen = pdmr_row['StandardizedRegimen'][0]
            if regimen.__contains__('naive'):
                row['treatment_naive_at_collection'], row['treated_prior_to_collection'], row['treated_at_collection'] = 'Yes', 'No', 'Not provided' 
            else:
                row['treatment_naive_at_collection'], row['treated_prior_to_collection'], row['treated_at_collection'] = 'No', 'Yes', 'Not provided'
    final.append(row.tolist())
final = pd.DataFrame(final, columns=df.columns)

In [118]:
df = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-cell_model.tsv')
pdmr_specimen = pd.read_csv('/Users/tushar/Downloads/PDM_Samples (3).csv')
final = list()
for _, row in df.iterrows():
    if _ > 3:
        model_id = row['model_id']
        pdmr_row = pdmr_specimen[[True if model_id == r else False for r in pdmr_specimen['model_id']]].reset_index(drop=True)
        if len(pdmr_row)>0:
            row['growth_properties'], row['supplements'] = pdmr_row['GrowthProperties'][0], pdmr_row['RequiredMedia'][0] 
    final.append(row.tolist())
final = pd.DataFrame(final, columns=df.columns)

In [120]:
final.to_csv('/Users/tushar/PDMR/PDMR_metadata-cell_model.tsv', sep='\t', index=False)

In [287]:
pdmr_cell = pd.read_csv('/Users/tushar/Downloads/PDM_Samples (3).csv')
pdmr_pdx = pd.read_csv('/Users/tushar/Downloads/PDM_Samples (4).csv')
pdmr_patient = pd.read_csv('/Users/tushar/Downloads/PatientInformation.csv', encoding = "ISO-8859-1")
pdmr_sample = pd.read_csv('/Users/tushar/Downloads/Find Specimen.csv')
treatment = pd.read_csv('/Users/tushar/Downloads/PDM_TreatmentHistoryReport (1).csv')
pdmr_pdx['model_id'] = pdmr_pdx['Patient ID'] + '-' + pdmr_pdx['Specimen ID']
url_prefix = "https://pdmdb.cancer.gov/pls/apex/f?p=101:4:0::NO:4:P4_SPECIMENSEQNBR:"
pdmr_sample['sharing_url'] = url_prefix + pdmr_sample['View'].astype(str)
pdmr_ps = pdmr_sample.merge(pdmr_patient, on='Patient ID', how='left')

In [288]:
cm_cell = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-cell_model.tsv')
cm_pdx = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-pdx_model.tsv')
cm_p = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-patient.tsv')
cm_ps = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-patient_sample.tsv')
cm_mv = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-model_validation.tsv')
cm_sh = read_metadata_with_fields('/Users/tushar/PDMR/PDMR_metadata-sharing.tsv')

In [289]:
to_add_cell = pdmr_cell[[True if r not in cm_cell['model_id'].iloc[4:].to_list() else False for r in pdmr_cell['model_id'].to_list()]]
to_add_pdx = pdmr_pdx[[True if r not in cm_pdx['model_id'].iloc[4:].to_list() else False for r in pdmr_pdx['model_id'].to_list()]]
pdx_passage = {}
for _, r in to_add_pdx.groupby('model_id'):
    passages = ', '.join([str(int(p)) for p in set(r.Passage.fillna(0.0))])
    pdx_passage[r.model_id.iloc[0]] = passages

In [290]:
def return_ps_row(model_id, pdmr_ps, cm_ps, treatment):
    ps = pdmr_ps[[ True if model_id.__contains__(m) else False for m in pdmr_ps['model_id']]].reset_index(drop=True)
    tt = 'Metastatic' if ps['ProvidedTissue Origin'].__contains__('Metastatic') else 'Primary'
    patient_id = ps['Patient ID'][0]
    pdmr_row = treatment[[True if patient_id == r else False for r in treatment['Patient ID']]].reset_index(drop=True)
    if len(pdmr_row)>0:
        regimen = pdmr_row['StandardizedRegimen'][0]
        if regimen.__contains__('naive'):
            tnc, tpc, tac = 'Yes', 'No', 'Not provided' 
        else:
            tnc, tpc, tac = 'No', 'Yes', 'Not provided'
    ps_row = pd.DataFrame([
        ['', ps['Patient ID'][0], model_id, ps['Date ofDiagnosis'][0], '', '', 
        ps['Age atSampling'][0], ps['CTEP SDCDescription_x'][0], tt, ps['Disease BodyLocation_x'][0], ps['BiopsySite'][0], ps['TissueType'][0], '', '', '', '', 'Not provided', ps['Molecular andIHC Data'][0], 'Yes', tnc, tpc, tac, '', model_id]
    ], columns=cm_ps.columns)
    return ps_row
    
def construct_sharing_row(model_id, pdmr_ps, cm_sh):
    ps = pdmr_ps[[ True if model_id.__contains__(m) else False for m in pdmr_ps['model_id']]].reset_index(drop=True)
    row = pd.DataFrame([
        ['', model_id, 'Academia and Industry', '', '15/11/2024', 'available', 'hilfikerp@mail.nih.gov','Pete Hilfiker',
'https://pdmr.cancer.gov/request/default.htm', ps['sharing_url'][0], 'CC0']
    ], columns=cm_sh.columns)
    cm_sh = pd.concat([cm_sh, row])
    return cm_sh

for _, r in to_add_cell.iterrows():
    mt = 'Cell line' if r['PDM Type'].__contains__('PDC') else 'Organoid'
    ps_row = return_ps_row(r['model_id'], pdmr_ps, cm_ps, treatment)
    row = pd.DataFrame([
        ['', r['model_id'], '', '', mt, '', '', 
        r['GrowthProperties'], 'Invitrogen, Cat#: 12634-010', 'DMEM/F12', r['RequiredMedia'], 
        '', '', 'No', 'None', '10% FBS','Not provided', 'Not provided', '', 'PDMR', 'Academic', 
        r['model_id'], 'Not provided', 'Not provided', '', r['CultureDerivation']]
    ], columns=cm_cell.columns)
    
    mv_row = pd.DataFrame([
        ['', r['model_id'], 'Histology', 'Model validated against histological features of same diagnosis', 'All', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided']
    ], columns=cm_mv.columns)
    cm_mv = pd.concat([cm_mv, mv_row])
    cm_sh = construct_sharing_row(r['model_id'], pdmr_ps, cm_sh)
    cm_cell = pd.concat([cm_cell, row], ignore_index=True)
    cm_ps = pd.concat([cm_ps, ps_row])
    
for r in to_add_pdx['model_id'].unique():
    subset = pdmr_sample[pdmr_sample['model_id'] == r].reset_index(drop=True)
    ps_row = return_ps_row(r, pdmr_ps, cm_ps, treatment)
    cm_ps = pd.concat([cm_ps, ps_row])
    cm_sh = construct_sharing_row(r, pdmr_ps, cm_sh)
    
    es = subset['Viable PassageImplantation Site'][0]
    et = 'Heterotopic' if es.__contains__('flank') else 'Orthotopic'
    row = pd.DataFrame([
        ['', r, 'NSG', 'NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</sup>/SzJ', es, et, 
        'tissue fragment', 'Fresh', pdx_passage[r], 
        '', '', '', 'PDMR', 'Academic', r, 'Not provided', '']
    ], columns=cm_pdx.columns)
    cm_pdx = pd.concat([cm_pdx, row], ignore_index=True)
    mv_row = pd.DataFrame([
        ['', r, 'Histology', 'Model validated against histological features of same diagnosis', 'All', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided']
    ], columns=cm_mv.columns)
    cm_mv = pd.concat([cm_mv, mv_row])
    
    
for pid in cm_ps.patient_id.iloc[4:].unique():
    if pid in cm_p.patient_id[4:].unique():
        continue
    subset = pdmr_ps[pdmr_ps['Patient ID'] == pid].reset_index(drop=True).iloc[0]
    agecat = 'Adult' if int(subset['Age atDiagnosis']) > 21 else 'Pediatric'
    smok = 'Current smoker' if subset['Has Smoked100 Cigarettes'] == 'Yes' else 'Non smoker'
    row = pd.DataFrame([
        ['', pid, subset['BiologicalSex'], subset['PatientNotes'], subset['Race'], 'Not provided', subset['CTEP SDCDescription_y'], subset['Age atDiagnosis'], agecat, smok, 'Not provided', 'Not provided', 'Not provided']
    ], columns=cm_p.columns)
    cm_p = pd.concat([cm_p, row], ignore_index=True)
    
cm_cell.to_csv('/Users/tushar/PDMR/fixed/PDMR_metadata-cell_model.tsv', sep='\t', index=False)
cm_pdx.to_csv('/Users/tushar/PDMR/fixed/PDMR_metadata-pdx_model.tsv', sep='\t', index=False)
cm_mv.to_csv('/Users/tushar/PDMR/fixed/PDMR_metadata-model_validation.tsv', sep='\t', index=False)
cm_ps.to_csv('/Users/tushar/PDMR/fixed/PDMR_metadata-patient_sample.tsv', sep='\t', index=False)
cm_p.to_csv('/Users/tushar/PDMR/fixed/PDMR_metadata-patient.tsv', sep='\t', index=False)
cm_sh.to_csv('/Users/tushar/PDMR/fixed/PDMR_metadata-sharing.tsv', sep='\t', index=False)

In [263]:
cm_p

,Field,patient_id,sex,history,ethnicity,ethnicity_assessment_method,initial_diagnosis,age_at_initial_diagnosis,age_category,smoking_status,alcohol_status,alcohol_frequency,family_history_of_cancer
0,#Description,Unique anonymous/de-identified ID for the pati...,Sex of the patient,Cancer relevant comorbidity or environmental e...,Patient Ethnic group. Can be derived from self...,Patient Ethnic group assessment method. Leave ...,Diagnosis of the patient when first diagnosed ...,This is the age of first diagnosis. Can be pri...,Age category at time of sampling,Patient's smoking history,"Alcohol intake of the patient, self-reported",The average number of days per week on which t...,If a first-degree relative of the patient has ...
1,#Example,A0088,female,lynch syndrome,Eastern European,self-assessed/genetic,colorectal carcinoma,70,Adult,"current smoker, 1 pack per day",yes,3 days/week,yes
2,#Format Requirements,free alphanumerical and -_~.,"male,female,other,not collected, or not provided",free alphanumerical text,Use NCIT term where possible. Ctrl-Click here ...,free alphanumerical text,Use NCIT term when possible. Ctrl-Click here t...,Numerical. Can be exact age or binned in 10 ye...,"Adult, Pediatric, Fetus, Not collected, Not pr...","Current smoker, ex-smoker, non-smoker, never s...",yes/no/not provided,free alphanumerical,free text
3,#Essential?,essential,essential,desirable,desirable,desirable,desirable,desirable,desirable,desirable,desirable,desirable,desirable
4,NaN,523234,Female,Final Pathology: Dx confirmed. Immunophenoty...,Not provided,Not provided,Endometrioid endomet. adenocar.,72,Adult,Not Provided,Not Provided,Not Provided,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,NaN,K57222,Female,NaN,Not Provided,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not provided,Not provided,Not provided
934,NaN,939949,Male,Current Smoker,Not Hispanic or Latino,Not provided,Not provided,Not Provided,Not Provided,Current Smoker,Not provided,Not provided,Not provided
935,NaN,K42656,Male,NaN,Not Provided,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not provided,Not provided,Not provided
936,NaN,926114,Female,NaN,Not Provided,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not provided,Not provided,Not provided


In [265]:
for pid in cm_ps.patient_id.iloc[4:].unique():
    if pid in cm_p.patient_id[4:].unique():
        continue
    subset = pdmr_ps[pdmr_ps['Patient ID'] == pid].reset_index(drop=True).iloc[0]
    agecat = 'Adult' if int(subset['Age atDiagnosis']) > 21 else 'Pediatric'
    smok = 'Current smoker' if subset['Has Smoked100 Cigarettes'] == 'Yes' else 'Non smoker'
    row = pd.DataFrame([
        ['', pid, subset['BiologicalSex'], subset['PatientNotes'], subset['Race'], 'Not provided', subset['CTEP SDCDescription_y'], subset['Age atDiagnosis'], agecat, smok, 'Not provided', 'Not provided', 'Not provided']
    ], columns=cm_p.columns)
    cm_p = pd.concat([cm_p, row], ignore_index=True)


,Field,patient_id,sex,history,ethnicity,ethnicity_assessment_method,initial_diagnosis,age_at_initial_diagnosis,age_category,smoking_status,alcohol_status,alcohol_frequency,family_history_of_cancer
0,,147741,Female,Tumor Stage/Grade: High Grade; Stage IVb \nLoc...,White,Not provided,Ovarian epithelial cancer,48,Adult,Non smoker,Not provided,Not provided,Not provided


In [21]:
mut_tsv = read_metadata_with_fields(join(final_path, "mut/PDMR_mut.tsv"))
mut_og = read_metadata_with_fields(join(home, "PDMR/mut/PDMR_mut.tsv"))
for _, row in mut_tsv.iterrows():
    if row['sample_id'] in mut_og.sample_id.unique():
        continue
    mut_og = pd.concat([mut_og, pd.DataFrame(row).transpose()], ignore_index=True)
mut_og

,sample_id,symbol,biotype,coding_sequence_change,variant_class,codon_change,amino_acid_change,consequence,functional_prediction,read_depth,...,seq_start_position,ref_allele,alt_allele,ucsc_gene_id,ncbi_gene_id,ncbi_transcript_id,ensembl_gene_id,ensembl_transcript_id,variation_id,platform_id
0,455876-151-R-V1-organoid,FLACC1,protein_coding,NaN,SNV,NaN,NaN,downstream_gene_variant,NaN,230,...,201285170,C,G,NaN,NaN,NaN,ENSG00000155749,ENST00000392257,rs770193825&COSV51847483&COSV51851164,mutation_OncoKB_Gene_Panel
1,455876-151-R-V1-organoid,PIK3CA,protein_coding,1090G>A,SNV,Gga/Aga,G364R,missense_variant,NaN,96,...,179204533,G,A,NaN,NaN,NaN,ENSG00000121879,ENST00000263967,rs1576935161&CM126689&COSV55903618,mutation_OncoKB_Gene_Panel
2,455876-151-R-V1-organoid,CDKN2A,protein_coding,358G>T,SNV,Gag/Tag,E120*,stop_gained,NaN,107,...,21971001,C,A,NaN,NaN,NaN,ENSG00000147889,ENST00000304494,rs1819691962&CD972119&COSV58683444&COSV58717600,mutation_OncoKB_Gene_Panel
3,455876-151-R-V1-organoid,AJUBA,protein_coding,383dup,insertion,tcg/tCcg,NaN,frameshift_variant,NaN,89,...,22981884,G,GG,NaN,NaN,NaN,ENSG00000129474,ENST00000262713,NaN,mutation_OncoKB_Gene_Panel
4,455876-151-R-V1-organoid,TP53,protein_coding,718A>G,SNV,Agt/Ggt,S240G,missense_variant,NaN,148,...,7674245,T,C,NaN,NaN,NaN,ENSG00000141510,ENST00000269305,rs1567549584&CM169323&COSV52677032&COSV5309743...,mutation_OncoKB_Gene_Panel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21000,K69424-012-T-V1-organoid,TSC2,NaN,5068+27_5069-47del,NaN,NaN,NaN,NaN,NaN,183,...,2087924,CCCTGCAGTGCAGGAAAGGTAGGGCCGGGTGGGG,-,NaN,NaN,NaN,NaN,NaN,NaN,mutation_OncoKB_Gene_Panel
21001,528814-157-R-AU3M947,APC,NaN,646C>T,NaN,NaN,NaN,NaN,NaN,57,...,112792446,C,T,NaN,NaN,NaN,NaN,NaN,NaN,mutation_OncoKB_Gene_Panel
21002,528814-157-R-N0W,APC,NaN,646C>T,NaN,NaN,NaN,NaN,NaN,55,...,112792446,C,T,NaN,NaN,NaN,NaN,NaN,NaN,mutation_OncoKB_Gene_Panel
21003,528814-157-R-N1WE11G11,APC,NaN,646C>T,NaN,NaN,NaN,NaN,NaN,48,...,112792446,C,T,NaN,NaN,NaN,NaN,NaN,NaN,mutation_OncoKB_Gene_Panel


In [25]:
read_metadata_with_fields(join(home, "PDMR/mut/PDMR_mut.tsv")).shape[0] +  mut_tsv.shape[0]

52262

In [23]:
mut_tsv.shape

(33519, 23)

In [26]:
mut_og.to_csv(join(home, "PDMR/mut/PDMR_mut.tsv"), sep='\t', index=False)

In [57]:
[s for s in sample_mut if s.__contains__('organoid-')]

[]

In [55]:
pdmr_mut = read_metadata_with_fields(join(home, "PDMR/mut/PDMR_mut.tsv"))
pdmr_mut['sample_id'] = pdmr_mut['sample_id'].str.replace('organoid-.*', 'organoid', regex=True) 

pdmr_mms = read_metadata_with_fields(join(home, "PDMR/PDMR_molecular_metadata-sample.tsv"))
pdmr_pdx = read_metadata_with_fields(join(home, "PDMR/PDMR_metadata-pdx_model.tsv"))
pdmr_mms_subset = pdmr_mms[ pdmr_mms.platform_id.str.contains('mut')]
sample_mut = pdmr_mut.sample_id.unique()
for s in sample_mut:
    if s in pdmr_mms_subset.sample_id.unique():
        continue
    passage = ''
    hs = ''
    hsn = ''

    if s.__contains__('organoid') or s.__contains__('PDC'):
        model_id = s
        model_type = 'cell'
    else:
        model_id = '-'.join(s.split('-')[:3])
        temp = pdmr_pdx[pdmr_pdx['model_id'] == model_id].reset_index(drop=True)
        try:
            passage = temp['passage_number'][0]
            hs = temp['host_strain_name'][0]
            hsn = temp['host_strain_nomenclature'][0]
        except:
            print(model_id, s)
        model_type =  'patient' if s == model_id else 'xenograft'
    platform_id = 'mutation_OncoKB_Gene_Panel'
    row = ['', model_id, s, model_type, passage, hs, hsn, '', '', platform_id]
    pdmr_mms = pd.concat([pdmr_mms, pd.DataFrame([row], columns=pdmr_mms.columns)], ignore_index=True)
pdmr_mms

BL0479-F1894-M22M200M316 BL0479-F1894-M22M200M316
BL0479-F1894-M22 BL0479-F1894-M22
BL0479-F1894-M23M205M318 BL0479-F1894-M23M205M318
BL0479-F1894-M23M205 BL0479-F1894-M23M205
BL0479-F1894-M23M207M324M413 BL0479-F1894-M23M207M324M413
BL0479-F1894-M23 BL0479-F1894-M23
BL0479-F1894-M24M216 BL0479-F1894-M24M216


,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,free alphanumerical,free alphanumerical
3,#Essential?,essential,essential,essential,essential,essential,essential,recommended,desirable,essential
4,NaN,328634-206-R-V1-organoid,328634-206-R-V1-organoid,cell,NaN,NaN,NaN,NaN,NaN,mutation_OncoKB_Gene_Panel
...,...,...,...,...,...,...,...,...,...,...
22904,,K69424-012-T-V1-organoid,K69424-012-T-V1-organoid,cell,,,,,,mutation_OncoKB_Gene_Panel
22905,,528814-157-R,528814-157-R-AU3M947,xenograft,"0,1,2",NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,,,mutation_OncoKB_Gene_Panel
22906,,528814-157-R,528814-157-R-N0W,xenograft,"0,1,2",NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,,,mutation_OncoKB_Gene_Panel
22907,,528814-157-R,528814-157-R-N1WE11G11,xenograft,"0,1,2",NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,,,mutation_OncoKB_Gene_Panel


In [61]:
pdmr_mms.to_csv(join(home, "PDMR/PDMR_molecular_metadata-sample.tsv"), sep='\t', index=False)

In [41]:
pdmr_mms

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,free alphanumerical,free alphanumerical
3,#Essential?,essential,essential,essential,essential,essential,essential,recommended,desirable,essential
4,NaN,328634-206-R-V1-organoid,328634-206-R-V1-organoid,cell,NaN,NaN,NaN,NaN,NaN,mutation_OncoKB_Gene_Panel
...,...,...,...,...,...,...,...,...,...,...
20932,NaN,K93363-136-R,K93363-136-R,patient,NaN,NaN,NaN,NaN,NaN,immune_HLA_type
20933,NaN,K93363-136-R,K93363-136-R,patient,NaN,NaN,NaN,NaN,NaN,immune_mmr
20934,NaN,K93363-136-R,K93363-136-R,patient,NaN,NaN,NaN,NaN,NaN,immune_tmb
20935,NaN,K93363-136-R,K93363-136-R,patient,NaN,NaN,NaN,NaN,NaN,immune_mpm


In [64]:
temp

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,,111316-319-R,111316-319-R-AH5Q86T81,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
1,,111316-319-R,111316-319-R-AH5Q89,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
2,,111316-319-R,111316-319-R-AH6T26,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
3,,111316-319-R,111316-319-R-AH8,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
4,,111316-319-R,111316-319-R-AH8Q94,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
...,...,...,...,...,...,...,...,...,...,...
2071,,K93363-136-R,K93363-136-R-KYAE34E63,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
2072,,K93363-136-R,K93363-136-R-KYCE48,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
2073,,K93363-136-R,K93363-136-R-KYE,xenograft,,,,,,expression_RNASEQ_Illumina_NGS
2074,,K93363-136-R-V1-organoid,K93363-136-R-V1-organoid,cell,,,,,,expression_RNASEQ_Illumina_NGS


In [66]:
pd.concat([pdmr_mms, temp]).to_csv(join(home, "PDMR/PDMR_molecular_metadata-sample.tsv"), sep='\t', index=False)

In [74]:
treatment_update = read_metadata_with_fields(join(update_path, 'treatment/patient-treatment.tsv')).iloc[4:]
treatment_og = read_metadata_with_fields(join(home, 'PDMR/treatment/PDMR_patienttreatment-Sheet1.tsv'))
subset_model_id_og = treatment_og.model_id.unique()
treatment_update = treatment_update[[True if m not in subset_model_id_og else False for m in treatment_update.model_id]]

In [78]:
pd.concat([treatment_og, treatment_update], ignore_index=True).to_csv(join(home, 'PDMR/treatment/PDMR_patienttreatment-Sheet1.tsv'), sep='\t', index=False)

In [77]:
treatment_og

,Field,patient_id,treatment_name,treatment_type,treatment_dose,treatment_starting_date,treatment_duration,treatment_event,elapsed_time,treatment_response,response_classification,model_id
0,#Description,Unique anonymous/de-identified provider ID for...,"treatment name, can be surgery, radotherapy, ...","Treatment type categorised as Surgery, Radiati...",Treatment dose and unit,treatement starting date in month and year - t...,treatment duration,IF DATES ARE ENTERED IN PREVIOUS COLUMN - DON...,IF TREATMENT DATES ARE ENTERED IN PREVIOUS COL...,reponse of prior treatment treatment,classification used to define response to trea...,Unique identifier for PDX when treatment was c...
1,#Example,CRC0014LM,xenograft,Chemotherapy,7,Jul-09,54,NaN,NaN,progressive disease,immunohistochemistry,CRL-240
2,#Format Requirements,unique for a patient,list drugs or conpounds or suregry or radiothe...,"[Surgery, Radiation Therapy, Chemotherapy, Imm...",add the dose and corresponding units,starting date when patient was treated,in months,combination between event and a number,difference between current event and 1st even...,when possible give response based on RECIST c...,classification used to define response to trea...,this needs to be unique to represent an aggreg...
3,#Essential?,essential,essential,essential,desirable,essential,desirable,if no dates in treatment starting dates this i...,if no dates in treatment starting dates this i...,desirable,desirable,essential
4,NaN,535342,Anastrozole,Hormone therapy,Not provided,Not provided,4,NaN,NaN,Not provided,Not provided,535342-205-R
...,...,...,...,...,...,...,...,...,...,...,...,...
1285,NaN,LG0703,Erlotinib,Targeted therapy,Not provided,Not provided,Not provided,NaN,NaN,Not provided,Not provided,LG0703-F948
1286,NaN,419622,Fluorouracil+Cisplatin,Chemotherapy,Not provided,Not provided,2,NaN,NaN,progressive disease,Not provided,419622-098-T-V1-organoid
1287,NaN,136153,Radiation Exposure,Radiation therapy,Not provided,Not provided,0,NaN,NaN,Complete Response,Not provided,136153-299-R
1288,NaN,136153,Tamoxifen,Hormone therapy,Not provided,Not provided,84,NaN,NaN,Complete Response,Not provided,136153-299-R


In [ ]:
|